In [7]:
import pandas as pd
import sqlite3
import json


In [12]:
orders = pd.read_csv('order.csv')
orders.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [13]:
users = pd.read_json('users.json')
users.head()

,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [14]:
with open('restaurants.sql', 'r') as f:
    sql_script = f.read()

conn = sqlite3.connect(":memory:")
conn.executescript(sql_script)

restaurants = pd.read_sql_query("SELECT * FROM restaurants", conn)
restaurants.head()

,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [15]:
orders.columns = orders.columns.str.lower()
users.columns = users.columns.str.lower()
restaurants.columns = restaurants.columns.str.lower()

In [16]:
df = orders.merge(users, how='left', on='user_id')
df = df.merge(restaurants, how='left', on='restaurant_id')

df.head(40)

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0
5,6,2933,499,12-07-2023,1497.22,Annapurna Tiffins South Indian,User_2933,Chennai,Regular,Restaurant_499,Mexican,4.9
6,7,970,35,30-05-2023,129.21,Royal Biryani North Indian,User_970,Bangalore,Regular,Restaurant_35,Chinese,3.9
7,8,891,57,07-11-2023,269.19,Spice Mess Punjabi,User_891,Pune,Regular,Restaurant_57,Mexican,4.4
8,9,364,7,05-12-2023,953.30,Ruchi Biryani Punjabi,User_364,Chennai,Gold,Restaurant_7,Italian,3.2
9,10,2972,183,30-12-2023,351.41,Taste of Biryani Non-Veg,User_2972,Bangalore,Regular,Restaurant_183,Chinese,3.7


In [18]:
df['order_date'] = pd.to_datetime(df['order_date'], dayfirst=True)


In [19]:
#Which city has the highest total revenue (total_amount) from Gold members?
df[df['membership']=='Gold'].groupby('city')['total_amount'].sum().sort_values(ascending=False)

city
Chennai      1080909.79
Pune         1003012.32
Bangalore     994702.59
Hyderabad     896740.19
Name: total_amount, dtype: float64

In [20]:
#Which cuisine has the highest average order value across all orders?
df.groupby('cuisine')['total_amount'].mean().sort_values(ascending=False)

cuisine
Mexican    808.021344
Italian    799.448578
Indian     798.466011
Chinese    798.389020
Name: total_amount, dtype: float64

In [21]:
#How many distinct users placed orders worth more than ₹1000 in total (sum of all their orders)?
user_spend = df.groupby('user_id')['total_amount'].sum()
(user_spend > 1000).sum()

np.int64(2544)

In [23]:
#Which restaurant rating range generated the highest total revenue?
bins = [3.0, 3.5, 4.0, 4.5, 5.0]
labels = ["3.0–3.5", "3.6–4.0", "4.1–4.5", "4.6–5.0"]

df['rating_range'] = pd.cut(df['rating'], bins=bins, labels=labels, include_lowest=True)

df.groupby('rating_range', observed=False)['total_amount'].sum().sort_values(ascending=False)


rating_range
4.6–5.0    2197030.75
3.0–3.5    2136772.70
4.1–4.5    1960326.26
3.6–4.0    1717494.41
Name: total_amount, dtype: float64

In [24]:
#Among Gold members, which city has the highest average order value?
df[df['membership']=='Gold'].groupby('city')['total_amount'].mean().sort_values(ascending=False)


city
Chennai      808.459080
Hyderabad    806.421034
Bangalore    793.223756
Pune         781.162243
Name: total_amount, dtype: float64

In [25]:
#Which cuisine has the lowest number of distinct restaurants but still contributes significant revenue?
# Number of restaurants per cuisine
restaurants.groupby('cuisine')['restaurant_id'].nunique().sort_values()

# Revenue per cuisine
df.groupby('cuisine')['total_amount'].sum().sort_values(ascending=False)

cuisine
Mexican    2085503.09
Italian    2024203.80
Indian     1971412.58
Chinese    1930504.65
Name: total_amount, dtype: float64

In [26]:
#What percentage of total orders were placed by Gold members? (Rounded to nearest integer)
round((df[df['membership']=='Gold'].shape[0] / df.shape[0]) * 100)

50

In [27]:
df[df['restaurant_name_y'] == 'Restaurant_294'][
    ['restaurant_name_x', 'cuisine', 'rating', 'total_amount']
].head()

,restaurant_name_x,cuisine,rating,total_amount
1407,Hotel Dhaba Multicuisine,Italian,3.1,817.71
1643,Hotel Dhaba Multicuisine,Italian,3.1,1327.91
2426,Hotel Dhaba Multicuisine,Italian,3.1,629.97
3174,Hotel Dhaba Multicuisine,Italian,3.1,1004.28
3243,Hotel Dhaba Multicuisine,Italian,3.1,1189.93


In [28]:
#Which combination contributes the highest revenue?

df.groupby(['membership','cuisine'])['total_amount'].sum().sort_values(ascending=False)

membership  cuisine
Regular     Mexican    1072943.30
            Italian    1018424.75
Gold        Mexican    1012559.79
            Italian    1005779.05
Regular     Indian      992100.27
Gold        Indian      979312.31
            Chinese     977713.74
Regular     Chinese     952790.91
Name: total_amount, dtype: float64

In [29]:
#During which quarter of the year is the total revenue highest?
df['order_date'] = pd.to_datetime(df['order_date'], dayfirst=True)
df['quarter'] = df['order_date'].dt.quarter

df.groupby('quarter')['total_amount'].sum().sort_values(ascending=False)

quarter
3    2037385.10
4    2018263.66
1    2010626.64
2    1945348.72
Name: total_amount, dtype: float64

In [30]:
#How many total orders were placed by users with Gold membership?
df[df['membership'] == 'Gold'].shape[0]

4987

In [36]:
#What is the total revenue (rounded to nearest integer) generated from orders placed in Hyderabad city?
round(df[df['city'] == 'Hyderabad']['total_amount'].sum())

1889367

In [32]:
#How many distinct users placed at least one order?
df['user_id'].nunique()

2883

In [33]:
#What is the average order value (rounded to 2 decimals) for Gold members?
round(df[df['membership'] == 'Gold']['total_amount'].mean(), 2)

np.float64(797.15)

In [34]:
#How many orders were placed for restaurants with rating ≥ 4.5?
df[df['rating'] >= 4.5].shape[0]

3374

In [35]:
#How many orders were placed in the top revenue city among Gold members only?
df[(df['membership'] == 'Gold') & (df['city'] == 'Chennai')].shape[0]

1337